In [ ]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque

# Environment Parameters
rows, cols = 4, 4  # Matrix size
learning_rate = 0.01
discount_factor = 0.9
exploration_rate = 1.0
min_exploration_rate = 0.01
exploration_decay = 0.995
episodes = 5000
batch_size = 32
memory_size = 1000

# Matrix of availability (0 = free, 1 = occupied)
avail_matrix = np.array([
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [1, 0, 0, 1],
    [0, 0, 1, 0]
])

# Matrix of weights (higher values indicate higher cost to place an item)
weight_matrix = np.array([
    [3, 9, 2, 1],
    [5, 4, 8, 2],
    [6, 3, 2, 7],
    [1, 5, 9, 3]
])

# Items to be placed and their values
items = [10, 15, 8]  # Example values for m = 3 items
m = len(items)  # Number of items
available_cells = [(i, j) for i in range(rows) for j in range(cols) if avail_matrix[i, j] == 0]
n = len(available_cells)  # Number of available cells

# Convert (item_id, cell index) into an action index
def state_to_index(item_id, cell_index):
    return item_id * n + cell_index

# Convert action index back to (item_id, i, j)
def index_to_state(index):
    item_id = index // n
    cell_index = index % n
    return item_id, available_cells[cell_index]

# Reward function: Penalizes weight, rewards item value, avoids occupied cells
def reward_function(i, j, item_id):
    if avail_matrix[i, j] == 1:
        return -100  # Penalize occupied cells
    return items[item_id] - weight_matrix[i, j]  # Reward item value, penalize weight

# Neural Network for DQN
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Initialize Networks
state_size = n + m  # Available cells + items placed
action_size = m * n  # Choosing (item, cell)
policy_net = DQN(state_size, action_size)
target_net = DQN(state_size, action_size)
target_net.load_state_dict(policy_net.state_dict())
optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate)

# Experience Replay Buffer
memory = deque(maxlen=memory_size)

# Select action (item, cell) using ε-greedy
def select_action(state, exploration_rate):
    if random.uniform(0, 1) < exploration_rate:
        item_id = random.choice(range(m))  # Select an item
        cell_index = random.choice(range(n))  # Select an available cell
        return item_id, available_cells[cell_index]
    else:
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            q_values = policy_net(state_tensor)
        best_action = torch.argmax(q_values).item()
        return index_to_state(best_action)

# Train DQN using Replay Buffer
def train_dqn():
    if len(memory) < batch_size:
        return

    batch = random.sample(memory, batch_size)
    state_batch, action_batch, reward_batch, next_state_batch = zip(*batch)

    state_batch = torch.tensor(state_batch, dtype=torch.float32)
    action_batch = [state_to_index(item_id, available_cells.index((i, j))) for item_id, (i, j) in action_batch]
    action_batch = torch.tensor(action_batch, dtype=torch.int64)
    reward_batch = torch.tensor(reward_batch, dtype=torch.float32)
    next_state_batch = torch.tensor(next_state_batch, dtype=torch.float32)

    q_values = policy_net(state_batch).gather(1, action_batch.unsqueeze(1)).squeeze(1)
    with torch.no_grad():
        next_q_values = target_net(next_state_batch).max(1)[0]
    expected_q_values = reward_batch + (discount_factor * next_q_values)

    loss = F.mse_loss(q_values, expected_q_values)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Training loop
for episode in range(episodes):
    if len(available_cells) < m:
        break  # Not enough space for all items

    state = np.zeros(state_size)  # Initial state: empty matrix, no items placed
    placed_items = set()  # Track placed items

    for step in range(m):  # We must place m items
        item_id, (i, j) = select_action(state, exploration_rate)

        # Ensure item is not placed twice
        if item_id in placed_items:
            continue
        placed_items.add(item_id)

        reward = reward_function(i, j, item_id)

        # Update state representation
        next_state = state.copy()
        next_state[available_cells.index((i, j))] = 1

        # Save experience to memory
        memory.append((state, (item_id, (i, j)), reward, next_state))

        # Train model
        train_dqn()

        state = next_state

    # Decay exploration rate
    exploration_rate = max(min_exploration_rate, exploration_rate * exploration_decay)

    # Update target network every 50 episodes
    if episode % 50 == 0:
        target_net.load_state_dict(policy_net.state_dict())

# Find the best placement
state = np.zeros(state_size)
state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
q_values = policy_net(state_tensor)
best_action = torch.argmax(q_values).item()
best_item_id, best_cell = index_to_state(best_action)

print(f"Best placement: Item {best_item_id} at {best_cell}")


Available cells: [(0, 0), (0, 2), (0, 3), (1, 0), (1, 1), (1, 3), (2, 1), (2, 2), (3, 0), (3, 1), (3, 3)]


/tmp/ipykernel_258675/2566880433.py:104: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  state_batch = torch.tensor(state_batch, dtype=torch.float32)


Best placement: Item 2 at (3, 3)
